In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [2]:
# in-sample percent survivors correctly predicted (on training set) 
# in-sample percent fatalities correctly predicted (on training set) 
# out-of-sample percent survivors correctly predicted (on test set) 
# out-of-sample percent fatalities correctly predicted (on test set)

def fit_and_check_performance(mlpc_obj: MLPClassifier, train_X, train_y, test_X, text_y):
    mlpc_obj.fit(train_X, train_y)
    
    # in-sample
    in_sample_predict_y = mlpc_obj.predict(train_X)
    in_sample_survivors_correct_rate = \
        1 - sum(in_sample_predict_y[train_y == 1] ^ train_y[train_y == 1]) / len(train_y[train_y == 1])
    in_sample_fatalities_correct_rate = \
        1 - sum(in_sample_predict_y[train_y == 0] ^ train_y[train_y == 0]) / len(train_y[train_y == 0])
    
    # out-sample
    out_sample_predict_y = mlpc_obj.predict(test_X)
    out_sample_survivors_correct_rate = \
        1 - sum(out_sample_predict_y[test_y == 1] ^ test_y[test_y == 1]) / len(test_y[test_y == 1])
    out_sample_fatalities_correct_rate = \
        1 - sum(out_sample_predict_y[test_y == 0] ^ test_y[test_y == 0]) / len(test_y[test_y == 0])
    
    print("in-sample percent survivors correctly predicted:     {:.2f}%".format(100*in_sample_survivors_correct_rate))
    print("in-sample percent fatalities correctly predicted:    {:.2f}%".format(100*in_sample_fatalities_correct_rate))
    print("out-of-sample percent survivors correctly predicted: {:.2f}%".format(100*out_sample_survivors_correct_rate))
    print("out-of-sample percent fatalities correctly predicted:{:.2f}%".format(100*out_sample_fatalities_correct_rate))
    
#     print(classification_report(in_sample_predict_y, train_y))
#     print(classification_report(out_sample_predict_y, test_y))

In [3]:
raw_data = pd.read_csv("Titanic.csv")
raw_data.head()

,Unnamed: 0,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1st,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.337494,B5,Southampton,2,NaN,"St Louis, MO"
1,2,1st,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.550003,C22 C26,Southampton,11,NaN,"Montreal, PQ / Chesterville, ON"
2,3,1st,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,4,1st,0,"Allison, Mr. Hudson Joshua Crei",male,30.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,5,1st,0,"Allison, Mrs. Hudson J C (Bessi",female,25.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [4]:
columns_to_use = ['pclass', 'sex', 'age', 'sibsp','survived']

data = raw_data.loc[:, columns_to_use]
data.head()

,pclass,sex,age,sibsp,survived
0,1st,female,29.0000,0,1
1,1st,male,0.9167,1,1
2,1st,female,2.0000,1,0
3,1st,male,30.0000,1,0
4,1st,female,25.0000,1,0


In [5]:
data.dropna(inplace=True)
data.head()

,pclass,sex,age,sibsp,survived
0,1st,female,29.0000,0,1
1,1st,male,0.9167,1,1
2,1st,female,2.0000,1,0
3,1st,male,30.0000,1,0
4,1st,female,25.0000,1,0


In [6]:
data['pclass'].unique()

array(['1st', '2nd', '3rd'], dtype=object)

In [7]:
data['sex'].unique()

array(['female', 'male'], dtype=object)

In [8]:
df = data.copy()
pclass_dict = {'1st': 1, '2nd': 2, '3rd':3}
sex_dict = {'male': 0, 'female':1}
df.loc[:, 'pclass'] = data['pclass'].apply(lambda x:pclass_dict[x])
df.loc[:, 'sex']    = data['sex'].apply(lambda x:sex_dict[x])
df.sample(10)

,pclass,sex,age,sibsp,survived
1287,3,0,51.0,0,0
172,1,0,50.0,0,0
700,3,0,17.0,0,0
745,3,1,30.0,0,1
96,1,0,50.0,1,0
127,1,1,35.0,1,1
714,3,0,32.0,0,1
24,1,1,29.0,0,1
158,1,0,55.0,0,0
1094,3,1,31.0,0,1


In [9]:
# split data into train set and test set

train, test= train_test_split(df, test_size=0.2)
train_X =np.array( train.loc[:, columns_to_use[:4]])
train_y =np.array( train.loc[:, columns_to_use[-1]])
test_X = np.array( test.loc[:, columns_to_use[:4]])
test_y = np.array( test.loc[:,columns_to_use[-1]])

In [10]:
scaler = StandardScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)

In [11]:
mlpc_obj = MLPClassifier(
    max_iter=2000,
    hidden_layer_sizes=(100, 100), 
    activation='logistic',
    solver='adam', 
    alpha=1e-5, 
    momentum=0.1
)
fit_and_check_performance(mlpc_obj, train_X, train_y, test_X, test_y)

in-sample percent survivors correctly predicted:     69.21%
in-sample percent fatalities correctly predicted:    85.45%
out-of-sample percent survivors correctly predicted: 65.12%
out-of-sample percent fatalities correctly predicted:84.68%


In [12]:
mlpc_obj = MLPClassifier(
    max_iter=2000,
    hidden_layer_sizes=(5, 5), 
    activation='logistic',
    solver='adam', 
    alpha=1e-5, 
    momentum=0.1
)
fit_and_check_performance(mlpc_obj, train_X, train_y, test_X, test_y)

in-sample percent survivors correctly predicted:     64.81%
in-sample percent fatalities correctly predicted:    91.11%
out-of-sample percent survivors correctly predicted: 58.14%
out-of-sample percent fatalities correctly predicted:93.55%


In [13]:
# weights
mlpc_obj.coefs_

[array([[-1.37307531, -1.24481443, -1.17506222,  0.74748225,  1.43487892],
        [ 1.19576842,  1.66559809,  1.93368452, -0.87653175, -1.16945585],
        [-0.49931116, -0.39428097, -0.57186456, -0.01981316,  0.56962063],
        [-0.21175227, -0.30130498, -0.29593796,  0.08795721,  0.23915882]]),
 array([[-0.80845782,  1.17329921,  1.10752007,  1.00063045,  1.01783103],
        [-1.58909006,  1.1095371 ,  1.14911694,  0.90937085,  0.88302393],
        [-1.77901204,  0.89297517,  1.22396054,  1.05708706,  1.53197268],
        [ 0.43967157, -0.95300701, -0.86815862, -0.67049758, -0.81615604],
        [ 1.90077647, -0.89969181, -1.6675493 , -1.81408251, -1.74197356]]),
 array([[-1.25982631],
        [ 0.58414214],
        [ 1.26554414],
        [ 0.78050012],
        [ 0.79365334]])]

In [14]:
# bias
mlpc_obj.intercepts_

[array([-1.05022046, -1.20301296, -1.08990657,  1.00362644,  1.03570401]),
 array([ 1.10009871, -0.66487258, -0.51645242, -0.31514452, -0.60310786]),
 array([-0.7073601])]

In [15]:
new_ wnp.array([[ 1.58225258, -1.05313641, -1.18466405, -1.03126613,  1.42927034],
       [-0.98301604,  1.19495354,  1.69552528,  1.24709011, -1.15521005],
       [ 0.58843179, -0.59370409, -0.78071255, -0.45506331,  0.65331011],
       [ 0.25715659, -0.34551263, -0.18852094, -0.18413947,  0.26986847]])

SyntaxError: invalid syntax (<ipython-input-15-dfbf27b9d6f5>, line 1)